Reference:

- https://neos-guide.org/content/quadratic-assignment-problem

In [9]:
using JuMP, GLPK

In [11]:
# flow matrix
F = [0 3 0 2;
     0 0 0 1;
     0 0 0 4;
     0 0 0 0]

4×4 Array{Int64,2}:
 0  3  0  2
 0  0  0  1
 0  0  0  4
 0  0  0  0

In [12]:
# distances matrix
D = [0  0 53   0;
     22 0 40   0;
     0  0  0  55;
     0  0  0   0]

4×4 Array{Int64,2}:
  0  0  53   0
 22  0  40   0
  0  0   0  55
  0  0   0   0

## Formulacion con constraints

In [40]:
model = Model(GLPK.Optimizer)

n = size(F, 1)
@variable(model, 0 <= x[1:n, 1:n] <= 1)

4×4 Array{VariableRef,2}:
 x[1,1]  x[1,2]  x[1,3]  x[1,4]
 x[2,1]  x[2,2]  x[2,3]  x[2,4]
 x[3,1]  x[3,2]  x[3,3]  x[3,4]
 x[4,1]  x[4,2]  x[4,3]  x[4,4]

Interpretamos $x_{ij}$ como $i$ siendo el indice de la facility y $j$ siendo el indice de la location. $i, j$ varian entre $1$ y $4$.

In [30]:
@constraint(model, sum(x, dims=1) .== 1)
@constraint(model, sum(x, dims=2) .== 1);

In [38]:
function costo(x, F, D) 
    n = size(x, 1)
    s = 0
    for i = 1:n
        for j = 1:n
            i == j && continue
            for h = 1:n
                for k = 1:n
                    k == h && continue
                    s += F[i, j] * D[h, k] * x[i, h] * x[j, k]
                end
            end
        end
    end
    s = s / 2
end

costo (generic function with 1 method)

In [39]:
s = costo(x, F, D)
@objective(model, Min, s)

LoadError: The solver does not support an objective function of type MathOptInterface.ScalarQuadraticFunction{Float64}.

In [ ]:
optimize!(model)

In [ ]:
termination_status(model)

## Formulacion como un problema QUBO

TO-DO